In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import csv
import os
 
main="D:/IoT Project/IoTScenarios"
fols=[main+"/"+i for i in os.listdir(main)]
files=[]
for i in fols:
    if("points" in i or "csv" in i):
      continue
    F=os.path.join(i,os.listdir(i)[0])
    while(F.find("conn.log.labeled")==-1):
        F=F+"//"+os.listdir(F)[0]
    files.append(F)
 
label='tunnel_parents   label   detailed-label'
j=0
Dict=dict()
li=[]
li2=[]
for fi in files:
    j=0
    tsv_file = open(fi)
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    for i in read_tsv:
        j=j+1
        if(j==7):
            li=i
        if(j==8):
            li2=i
            for key in range(1,len(li)):
                Dict[li[key]]=li2[key]
        if(j==9):
            break
 
frames=[]
j=0
print("All cols:",Dict.keys())
cols=['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label']
cols12 = ["ts","uid","id.orig_h","id.orig_p","id.resp_h","id.resp_p","proto","service","duration","orig_bytes","resp_bytes","conn_state","local_orig","local_resp","missed_bytes","history","orig_pkts","orig_ip_bytes","resp_pkts","resp_ip_bytes","label"]
print(cols)
for fi in files:
    df = pd.read_csv(fi, sep="\t",skiprows=10,names=cols12,nrows=10000)
    df.rename(columns={label:'label'},inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    df.label = df.label.apply(lambda x: 'B' if ('benign' in x or 'Benign' in x) else 'A')
    frames.append(df)
df=pd.concat(frames)
print(len(df))# nrows takes min(nrows,actual)

All cols: dict_keys(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents   label   detailed-label'])
['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label']
173084


In [8]:
df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1540469385.734089,CUrxU238nt0m6yTgKf,192.168.1.132,32893.0,216.239.35.8,123.0,udp,-,0.016986,48,...,SF,-,-,0.0,Dd,1.0,76.0,1.0,76.0,B
1,1540469831.302625,CGQf8t1kjdxB5PHXL4,192.168.1.132,53395.0,2.16.60.82,443.0,tcp,-,0.003497,0,...,SF,-,-,0.0,ShAFf,5.0,212.0,3.0,144.0,B
2,1540469831.265405,CUo9DH2QDnCaBIGjkg,192.168.1.132,52801.0,192.168.1.1,53.0,udp,dns,0.036724,34,...,SF,-,-,0.0,Dd,1.0,62.0,1.0,339.0,B
3,1540469418.379528,CAvXOZ3htimWEtglii,192.168.1.132,1900.0,239.255.255.250,1900.0,udp,-,384.518261,15072,...,S0,-,-,0.0,D,48.0,16416.0,0.0,0.0,B
4,1540470081.850824,CfJsUD2NGQvnK2p7Vd,192.168.1.132,58124.0,216.239.35.12,123.0,udp,-,0.270332,48,...,SF,-,-,0.0,Dd,1.0,76.0,1.0,76.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1532525402.008014,CREx2n1ZFaBYR8oWIl,192.168.100.111,26251.0,72.217.174.183,23.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,A
9995,1532525402.999198,CrVL3h4mABjmmviXta,192.168.100.111,22824.0,184.21.219.141,81.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,A
9996,1532525402.999202,CVhsq12NgjuRDxkoie,192.168.100.111,5065.0,66.144.41.159,81.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,A
9997,1532525402.999204,CjUCLy4VBB96jj5gol,192.168.100.111,47724.0,219.137.237.244,81.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,A


In [2]:
main_train=['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']
actual_col=df.columns
X_cols=['label']
for i in main_train:
    if(i in actual_col):
        X_cols.append(i)
 
print("X_cols",X_cols)
df.drop_duplicates(keep="first",inplace=True)
X_train=df[X_cols]
drop_cols=[]
output="label" 

temp_duration  = X_train.loc[X_train['duration']!='-','duration'].astype('float16')
index_inf_duration = temp_duration[temp_duration.isin([-np.inf, np.inf])]

temp_duration.drop([448,449], inplace = True)
avg_duration = np.asarray(temp_duration).mean()
X_train.loc[(X_train['duration']=='-'), 'duration'] = str(avg_duration )

X_train.drop([448,449], inplace = True)

X_train['duration'] = X_train['duration'].astype('float16')
X_train.reset_index(drop=True)
X_train=X_train.replace(to_replace ="-",value ="0")
Y = X_train[[output]]
Y_train=Y
X_train.drop(columns=[output], inplace=True)
X_train['duration'] = X_train['duration'].astype('float64')
X_train['orig_bytes'] = X_train['orig_bytes'].astype('float64')
X_train['resp_bytes'] = X_train['resp_bytes'].astype('float64')
X=X_train

X_cols ['label', 'duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']


c:\Users\rites\anaconda3\envs\IoT\lib\site-packages\pandas\core\dtypes\astype.py:170: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
C:\Users\rites\AppData\Local\Temp\ipykernel_17956\2716864825.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop([448,449], inplace = True)
C:\Users\rites\AppData\Local\Temp\ipykernel_17956\2716864825.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['duration'] = X_train['duration'].astype('float16')


In [3]:
import pandas as pd
import os
os.chdir('D:/IoT Project/IoTScenarios')
print("Change successful.")
import glob
file_names = [i for i in glob.glob('**/**/*')]
file_names
cols = ["ts","uid","id.orig_h","id.orig_p","id.resp_h","id.resp_p","proto","service","duration","orig_bytes","resp_bytes","conn_state","local_orig","local_resp","missed_bytes","history","orig_pkts","orig_ip_bytes","resp_pkts","resp_ip_bytes","label"]
iotdf = pd.DataFrame(columns=cols)
print(file_names)

Change successful.
['CTU-Honeypot-Capture-4-1\\bro\\conn.log.labeled', 'CTU-Honeypot-Capture-5-1\\bro\\conn.log.labeled', 'CTU-Honeypot-Capture-7-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-1-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-17-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-20-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-21-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-3-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-33-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-34-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-35-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-36-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-39-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-42-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-43-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-44-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-48-1\\bro\\conn.log.labeled', 'CTU-IoT-Malware-Capture-49-1\\bro\\conn.log.labeled', 'CTU

In [5]:

for file in file_names: 
  currdf = pd.read_csv(file,sep='\t',nrows=10000,names=cols,skiprows=10)
  currdf.drop(currdf.tail(1).index,inplace=True)
  iotdf = pd.concat([iotdf,currdf])
print("Concatenation complete.")
iotdf['service'].replace('-','new_service',inplace=True)
niotdf = iotdf.drop(columns=['ts','uid','local_orig','local_resp','missed_bytes'])

import numpy as np
niotdf.replace('-', np.nan,inplace=True)
ans = niotdf.groupby('label')
clean_niotdf = niotdf.apply(lambda x: x.fillna(x.value_counts().index[0]))
iotcp = clean_niotdf.copy()
iotcp = iotcp.replace(to_replace ='.*[bB]enign.*', value = 'Benign', regex = True)
iotcp = iotcp.replace(to_replace ='.*Malicious.*', value = 'Malicious', regex = True)
iotcp['duration'] = iotcp['duration'].astype('float64')
iotcp['orig_bytes'] = iotcp['orig_bytes'].astype('float64')
iotcp['resp_bytes'] = iotcp['resp_bytes'].astype('float64')
from sklearn.ensemble import RandomForestClassifier
import numpy as np
np.random.seed(0)
print(iotcp.columns)
iotcp = iotcp.drop(columns=['id.orig_h','id.resp_h'])
iotcp= pd.get_dummies(iotcp, columns=['proto'])
iotcp = pd.get_dummies(iotcp, columns=['conn_state'])
iotcp = pd.get_dummies(iotcp, columns=['service'])
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
y = iotcp['label']
x = iotcp.drop(columns=['label'])
# print(x.head())
Y=y
X=x

Concatenation complete.
Index(['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service',
       'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label'],
      dtype='object')
   id.orig_p  id.resp_p    duration  orig_bytes  resp_bytes history  \
0    32893.0      123.0    0.016986        48.0        48.0      Dd   
1    53395.0      443.0    0.003497         0.0         0.0   ShAFf   
2    52801.0       53.0    0.036724        34.0       311.0      Dd   
3     1900.0     1900.0  384.518261     15072.0         0.0       D   
4    58124.0      123.0    0.270332        48.0        48.0      Dd   

   orig_pkts  orig_ip_bytes  resp_pkts  resp_ip_bytes  ...  conn_state_SF  \
0        1.0           76.0        1.0           76.0  ...              1   
1        5.0          212.0        3.0          144.0  ...              1   
2        1.0           62.0        1.0          339.0  ...          

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
import time
from sklearn.preprocessing import RobustScaler

In [7]:
#Decision Tree

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, shuffle=True ,test_size=0.2)
DT = DecisionTreeClassifier()
start = time.time()
print('program start...')
print()
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
np.random.seed(0)
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size =0.3,stratify=Y,random_state=10)
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass
for column_name in xtest.columns:
        if xtest[column_name].dtype == object:
            print(column_name)
            xtest[column_name] = le.fit_transform(xtest[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytest=le.fit_transform(ytest)
ytrain=ydf

Y_test=ytest
X_test=xtest
X_train=xtrain
Y_train=ytrain

DT.fit(X_train, Y_train)
print()

print('prediction:')
y_pred = DT.predict(X_test)
train_pred=DT.predict(X_train)
print(train_pred[train_pred=='1'])
print(len(y_pred))
print("size of Y_test",len(Y_test))


print('Score:')
score = DT.score(X_test,Y_test)
print(score)

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

print("----Accuracy-----  ")
print(accuracy_score( Y_test , y_pred ))
print('----F1 score-----  ')
print(f1_score( Y_test , y_pred ))
print('----Recall------- ')
print(recall_score(Y_test , y_pred ))
print('----Precision----')
print(precision_score( Y_test , y_pred))

program start...

history
history

prediction:
[]
51926
size of Y_test 51926
Score:
0.9899279744251435
program end...

time cost: 
0.7532811164855957 seconds
----Accuracy-----  
0.9899279744251435
----F1 score-----  
0.993626381661528
----Recall------- 
0.9953609883536392
----Precision----
0.991897810218978


C:\Users\rites\AppData\Local\Temp\ipykernel_17956\854333036.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  print(train_pred[train_pred=='1'])


In [9]:
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytrain=ydf

In [10]:
path = DT.cost_complexity_pruning_path(X_train, Y_train)
path

{'ccp_alphas': array([0.00000000e+00, 1.37561421e-06, 1.37561421e-06, 1.37561421e-06,
        1.37561421e-06, 1.37561421e-06, 1.37561421e-06, 1.37561421e-06,
        1.37561421e-06, 1.37561421e-06, 1.71951776e-06, 2.06342132e-06,
        2.06342132e-06, 2.06342132e-06, 2.18479904e-06, 2.20098274e-06,
        2.20098274e-06, 2.20098274e-06, 2.20098274e-06, 2.20098274e-06,
        2.20098274e-06, 2.20098274e-06, 2.20098274e-06, 2.47610558e-06,
        2.75122842e-06, 2.75122842e-06, 2.75122842e-06, 2.75122842e-06,
        2.75122842e-06, 2.75122842e-06, 2.75122842e-06, 2.75122842e-06,
        2.75122842e-06, 2.75122842e-06, 2.75122842e-06, 2.75122842e-06,
        2.75122842e-06, 3.20976649e-06, 3.30147411e-06, 3.43903553e-06,
        3.43903553e-06, 3.49194377e-06, 3.61098731e-06, 3.61098731e-06,
        3.64133174e-06, 3.66830456e-06, 3.69243815e-06, 3.71415837e-06,
        3.89757360e-06, 4.12684264e-06, 4.12684264e-06, 4.12684264e-06,
        4.12684264e-06, 4.35611167e-06, 4.40196548

In [ ]:
#SVM
import sklearn.metrics as skm
print(df.shape)
print(X_cols)
#SVM_classifier
drop_cols=[]
SVM_classifier = SVC()
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
np.random.seed(0)
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size =0.3,stratify=Y,random_state=10)
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])
        else:
            pass
for column_name in xtest.columns:
        if xtest[column_name].dtype == object:
            print(column_name)
            xtest[column_name] = le.fit_transform(xtest[column_name])
        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytest=le.fit_transform(ytest)
ytrain=ydf

Y_test=ytest
X_test=xtest
X_train=xtrain
Y_train=ytrain
start = time.time()
print('program start...')
print()

SVM_classifier = SVC(C=1.0, cache_size=700, verbose=True).fit(X_train, Y_train.values.ravel())
print()
print(SVM_classifier.score(X_test, Y_test))

y_pred = SVM_classifier.predict(X_test)
wee=skm.accuracy_score(Y_test,y_pred,normalize=False)
print("Acuracy",wee)
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

print("Classifiction Report :")
print(classification_report(Y_test, y_pred))
print("----Accuracy-----  ")
print(accuracy_score( Y_test , y_pred ))
print('----F1 score-----  ')
print(f1_score( Y_test , y_pred))
print('----Recall------- ')
print(recall_score(Y_test , y_pred))
print('----Precision----')
print(precision_score( Y_test , y_pred))

In [11]:
#Rain Forest
#Rain forest is here
#import the necessary libraries
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
np.random.seed(0)
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size =0.3,stratify=Y,random_state=10)
clf = RandomForestClassifier(n_jobs=2,random_state=0)
ydf = pd.DataFrame(ytrain)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in xtrain.columns:
        if xtrain[column_name].dtype == object:
            #print(column_name)
            xtrain[column_name] = le.fit_transform(xtrain[column_name])

        else:
            pass
for column_name in xtest.columns:
        if xtest[column_name].dtype == object:
          xtest[column_name] = le.fit_transform(xtest[column_name])

        else:
            pass

ydf['label'] = le.fit_transform(ydf['label'])
ytest=le.fit_transform(ytest)
ytrain=ydf
clf.fit(xtrain,ytrain)
print(xtrain.shape)
predic_res = clf.predict(xtest)
ytestdf=ytest
print("----Accuracy-----  ")
print(accuracy_score( ytestdf , predic_res ))
print('----F1 score-----  ')
print(f1_score( ytestdf , predic_res))
print('----Recall------- ')
print(recall_score(ytestdf , predic_res))
print('----Precision----')
print(precision_score( ytestdf , predic_res))


C:\Users\rites\AppData\Local\Temp\ipykernel_17956\2110901112.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xtrain,ytrain)


(121158, 33)
----Accuracy-----  
0.9901590725262874
----F1 score-----  
0.9937732285383538
----Recall------- 
0.9956051468613424
----Precision----
0.991948039311083


In [17]:
#1D convnet

#Cnn
from sklearn import preprocessing
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
le = preprocessing.LabelEncoder()
Y=pd.DataFrame(Y)
Y=le.fit_transform(Y)
Y=np.asarray(Y).astype(np.int32)
X=pd.DataFrame(X)

X.history=le.fit_transform(X.history)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,stratify=Y, test_size=0.2)

# cnn model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import time
import tensorflow as tf

start = time.time()
tf.config.experimental_run_functions_eagerly(True)
print('program start...')
print()
siz_train=[X_train.shape[0],X_train.shape[1],1]
siz_test=[X_test.shape[0],X_test.shape[1],1]
for col in X_train:
  X_train[col] = np.asarray(X_train[col]).astype('float32')
  column = X_train[col]
  max_value = column.max()
  if(max_value>0):
    X_train[col] = X_train[col].apply(lambda x: x/max_value)
for col in X_test:
  X_test[col] = np.asarray(X_test[col]).astype('float32')
  column = X_test[col]
  max_value = column.max()
  if(max_value>0):
    X_test[col] = X_test[col].apply(lambda x: x/max_value)

x_train, x_test = X_train.values.reshape(siz_train), X_test.values.reshape(siz_test)
history = model.fit(x_train, Y_train, epochs = 20, batch_size=256, validation_data=(x_test,Y_test),verbose=1)
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

c:\Users\rites\anaconda3\envs\IoT\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
program start...

Epoch 1/20


C:\Users\rites\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


541/541 [==============================] - 33s 61ms/step - loss: 0.2558 - accuracy: 0.9110 - val_loss: 0.2421 - val_accuracy: 0.9147
Epoch 2/20
541/541 [==============================] - 33s 61ms/step - loss: 0.2435 - accuracy: 0.9145 - val_loss: 0.2390 - val_accuracy: 0.9148
Epoch 3/20
541/541 [==============================] - 33s 62ms/step - loss: 0.2384 - accuracy: 0.9147 - val_loss: 0.2333 - val_accuracy: 0.9150
Epoch 4/20
541/541 [==============================] - 33s 61ms/step - loss: 0.2315 - accuracy: 0.9149 - val_loss: 0.2241 - val_accuracy: 0.9151
Epoch 5/20
541/541 [==============================] - 33s 61ms/step - loss: 0.2239 - accuracy: 0.9148 - val_loss: 0.2228 - val_accuracy: 0.9150
Epoch 6/20
541/541 [==============================] - 33s 61ms/step - loss: 0.2167 - accuracy: 0.9155 - val_loss: 0.2137 - val_accuracy: 0.9150
Epoch 7/20
541/541 [==============================] - 33s 60ms/step - loss: 0.2126 - accuracy: 0.9205 - val_loss: 0.2126 - val_accuracy: 0.9154
Epo

In [18]:
model.save('IOT231dcnn')

INFO:tensorflow:Assets written to: IOT231dcnn\assets


INFO:tensorflow:Assets written to: IOT231dcnn\assets


In [19]:
print(history.history['accuracy'][-1],history.history['accuracy'][-2])

0.935443103313446 0.9348364472389221


In [20]:
while(history.history['accuracy'][-1]-history.history['accuracy'][-2]>0.001):
  history=model.fit(x_train, Y_train, epochs = 20, batch_size=256, validation_data=(x_test,Y_test),verbose=1)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# predict probabilities for test set
#yhat_probs = model.predict(X_test)
# predict crisp classes for test set
X_test=x_test
yhat_classes = model.predict(X_test)
yhat_classes = np.where(yhat_classes > 0.5, 1, 0)
# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
print(yhat_classes)
yhat_classes = yhat_classes
print(len(yhat_classes))
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_test, yhat_classes)
print('F1 score: %f' % f1)
#https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/
#https://stats.stackexchange.com/questions/258166/good-accuracy-despite-high-loss-value
#https://stats.stackexchange.com/questions/272754/how-do-you-interpret-the-cross-entropy-value

   1/1082 [..............................] - ETA: 1:34

C:\Users\rites\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1082/1082 [==============================] - 9s 8ms/step
[[1]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
34617
Accuracy: 0.935841
Precision: 0.927702
Recall: 0.996301
F1 score: 0.960778
